In [68]:
import copy

In [122]:
class Matrix(object):
    def __init__(self, data):
        n = len(data)
        m = len(data[0])
        assert all(len(sub_array) == m for sub_array in data)
        self.data = [[float(element) for element in row] for row in data]
        self.shape = (n, m)

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        return "Matrix([\n    " \
            + "\n    ".join(repr(row) for row in self.data) + "\n])"

    def __getitem__(self, item):
        return self.data[item]

    def copy(self):
        return Matrix(copy.deepcopy(self.data))

    def multiply_by(self, other):
        assert other.shape == tuple(reversed(self.shape))
        return Matrix([
            [
                float(sum(
                    self.data[self_row][self_column] \
                        * other.data[other_row][other_column]
                    for self_column, other_row in 
                        zip(range(self.shape[1]), range(other.shape[0]))
                ))
                for other_column in range(other.shape[1])
            ]
            for self_row in range(self.shape[0])
        ])

    @classmethod
    def identity(cls, n):
        matrix = cls([
            [0 for _ in range(n)]
            for _ in range(n)
        ])
        for j in range(n):
            matrix[j][j] = 1
        return matrix
        
    def is_square(self):
        return self.shape[0] == self.shape[1]

    def invert(self):
        n = self.shape[0]
        operations_so_far = Matrix.identity(n)
        self_copy = self.copy()

        for k in range(n):
            print(f"processing column {k}")
            print("self_copy =", self_copy)

            # reorder rows
            row_reordering = \
                list(range(k)) + \
                list(reversed(sorted(
                    range(k, n),
                    key=lambda j: abs(self_copy[j][k])
            )))
            reorder_op = Matrix([Matrix.identity(n).data[j] for j in row_reordering])
            print("reorder_op =", reorder_op)
            operations_so_far = reorder_op.multiply_by(operations_so_far)
#             print("operations_so_far =", operations_so_far)
            self_copy = reorder_op.multiply_by(self_copy)
            print("self_copy =", self_copy)

            # normalize this row
            normalization_op = Matrix.identity(n)
            normalization_op[k][k] = 1.0 / self_copy[k][k]
            print("normalization_op = ", normalization_op)
            operations_so_far = normalization_op.multiply_by(operations_so_far)
#             print("operations_so_far =", operations_so_far)
            self_copy = normalization_op.multiply_by(self_copy)
            print("self_copy =", self_copy)

            # eliminate the other rows
            eliminations_op = Matrix.identity(n)
            for j in range(n):
                if j == k:
                    continue
                eliminations_op[j][k] = -1.0 * self_copy[j][k]
            print("eliminations_op =", eliminations_op)
            operations_so_far = eliminations_op.multiply_by(operations_so_far)
            print("operations_so_far =", operations_so_far)
            self_copy = eliminations_op.multiply_by(self_copy)
            print("self_copy =", self_copy)

            print("\n")

        return operations_so_far


# Tests

## matrix inversion

In [123]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X_inverse = X.invert()
print("X_inverse = ", X_inverse)
X_inverse.multiply_by(X)

processing column 0
self_copy = Matrix([
    [3.0, 1.0]
    [4.0, 2.0]
])
reorder_op = Matrix([
    [0.0, 1.0]
    [1.0, 0.0]
])
self_copy = Matrix([
    [4.0, 2.0]
    [3.0, 1.0]
])
normalization_op =  Matrix([
    [0.25, 0.0]
    [0.0, 1]
])
self_copy = Matrix([
    [1.0, 0.5]
    [3.0, 1.0]
])
eliminations_op = Matrix([
    [1, 0.0]
    [-3.0, 1]
])
operations_so_far = Matrix([
    [0.0, 0.25]
    [1.0, -0.75]
])
self_copy = Matrix([
    [1.0, 0.5]
    [0.0, -0.5]
])


processing column 1
self_copy = Matrix([
    [1.0, 0.5]
    [0.0, -0.5]
])
reorder_op = Matrix([
    [1.0, 0.0]
    [0.0, 1.0]
])
self_copy = Matrix([
    [1.0, 0.5]
    [0.0, -0.5]
])
normalization_op =  Matrix([
    [1, 0.0]
    [0.0, -2.0]
])
self_copy = Matrix([
    [1.0, 0.5]
    [0.0, 1.0]
])
eliminations_op = Matrix([
    [1, -0.5]
    [0.0, 1]
])
operations_so_far = Matrix([
    [1.0, -0.5]
    [-2.0, 1.5]
])
self_copy = Matrix([
    [1.0, 0.0]
    [0.0, 1.0]
])


X_inverse =  Matrix([
    [1.0, -0.5]
    [-2.

Matrix([
    [1.0, 0.0]
    [0.0, 1.0]
])

In [78]:
Matrix([
    [1, 2, 0],
    [2, 1, 0],
    [3, 0, 1]
]).invert()

new_row_ordering [2, 1, 0]
result = Matrix([
    [0, 0, 1]
    [0, 1, 0]
    [1, 0, 0]
])
self_changed = Matrix([
    [3.0, 0.0, 1.0]
    [2.0, 1.0, 0.0]
    [1.0, 2.0, 0.0]
])
new_row_ordering [0, 2, 1]
result = Matrix([
    [0, 0, 1]
    [1, 0, 0]
    [0, 1, 0]
])
self_changed = Matrix([
    [3.0, 0.0, 1.0]
    [1.0, 2.0, 0.0]
    [2.0, 1.0, 0.0]
])
new_row_ordering [0, 1, 2]
result = Matrix([
    [0, 0, 1]
    [1, 0, 0]
    [0, 1, 0]
])
self_changed = Matrix([
    [3.0, 0.0, 1.0]
    [1.0, 2.0, 0.0]
    [2.0, 1.0, 0.0]
])


Matrix([
    [0, 0, 1]
    [1, 0, 0]
    [0, 1, 0]
])

In [54]:
Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3]
]).invert()

[0, 2, 1]
[0, 1, 2]
[0, 1, 2]


Matrix([
    [1, 0, 0]
    [0, 1, 0]
    [0, 0, 1]
])

In [48]:
Matrix([
    [1, 0],
    [0, 1]
]).invert()

Matrix([
    [1, 0]
    [0, 1]
])
Matrix([
    [1, 0]
    [0, 1]
])
Matrix([
    [1, 0]
    [0, 1]
])


Matrix([
    [1, 0]
    [0, 1]
])

In [32]:
Matrix([
    [2, 0],
    [0, 1]
]).invert()

Matrix([
    [1, 0]
    [0, 1]
])


Matrix([
    [1, 0]
    [0, 1]
])

In [33]:
Matrix([
    [1, 0],
    [0, 2]
]).invert()

Matrix([
    [0, 1]
    [1, 0]
])


Matrix([
    [0, 1]
    [1, 0]
])

In [36]:
Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 1]
]).invert().multiply_by(Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 1]
]))

Matrix([
    [0, 1, 0]
    [0, 0, 1]
    [1, 0, 0]
])


Matrix([
    [0.0, 2.0, 0.0]
    [0.0, 0.0, 1.0]
    [1.0, 0.0, 0.0]
])

## matrix multiplication


In [100]:
Matrix([[0.5]]).multiply_by(Matrix([[2]]))

Matrix([
    [1.0]
])

In [101]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 1]
]))

Matrix([
    [1.0, 2.0]
    [3.0, 4.0]
])

In [102]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [0, 1],
    [1, 0]
]))

Matrix([
    [2.0, 1.0]
    [4.0, 3.0]
])

In [103]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [2, 0],
    [0, 2]
]))

Matrix([
    [2.0, 4.0]
    [6.0, 8.0]
])

In [104]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 2]
]))

Matrix([
    [1.0, 4.0]
    [3.0, 8.0]
])

In [ ]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 1],
    [0, 1]
]))

### to do - matrix transpose

# Basics

In [ ]:
X = Matrix([[1, 2], [3, 4], [5, 6]])
X

In [ ]:
Y = Matrix([[1, 2, 3], [4, 5, 6]])
Y

In [ ]:
Y[1, 2:3, :]